In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from utils import get_face_landmarks

data_dir = 'faces/'  # Folder dataset

# ImageDataGenerator untuk augmentasi
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

output = []
for emotion_indx, emotion in enumerate(sorted(os.listdir(data_dir))):
    emotion_folder = os.path.join(data_dir, emotion)
    for image_path_ in os.listdir(emotion_folder):
        image_path = os.path.join(emotion_folder, image_path_)
        image = cv2.imread(image_path)

        face_landmarks = get_face_landmarks(image)
        
        if len(face_landmarks) == 1404:
            face_landmarks.append(int(emotion_indx))
            output.append(face_landmarks)

            # Augmentasi data (untuk dataset yang lebih sedikit)
            image = np.expand_dims(image, 0)
            i = 0
            for batch in datagen.flow(image, batch_size=1):
                aug_image = batch[0].astype('uint8')
                aug_face_landmarks = get_face_landmarks(aug_image)
                if len(aug_face_landmarks) == 1404:
                    aug_face_landmarks.append(int(emotion_indx))
                    output.append(aug_face_landmarks)
                i += 1
                if i >= 10:  # Membatasi augmentasi menjadi 10 gambar per gambar asli
                    break

np.savetxt('data_augmented.txt', np.asarray(output))


c:\Users\kuzup\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [16]:
from sklearn.ensemble import RandomForestClassifier
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Load augmented data from text file
data_file = "data_augmented.txt"
data = np.loadtxt(data_file)

# Split data into features (X) and labels (y)
X = data[:, :-1]
y = data[:, -1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

# Initialize Random Forest Classifier with class_weight
rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=30, class_weight='balanced')

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(confusion_matrix(y_test, y_pred))

# Save the model to a file
with open('model/model_file_augmented.pkl', 'wb') as f:
    pickle.dump(rf_classifier, f)


Accuracy: 73.03%
[[128  52  21   3  22]
 [ 45 225  41   5   7]
 [ 27  81 120   6   4]
 [  6  10  10 327   6]
 [ 10  16   5  10 248]]
